Model loading

In [1]:
from xcodec2.modeling_xcodec2 import XCodec2Model
 
model_path = "/home/uetuluk/hkust-audio-models/models/xcodec2"  
 
Codec_model = XCodec2Model.from_pretrained(model_path)
Codec_model.eval()
Codec_model.to("cpu")

/home/uetuluk/miniconda3/envs/xcodec2-infer-lib-buildtest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using a model of type xcodec2 to instantiate a model of type bigcodec. This is not supported for all configurations of models and can yield errors.
/home/uetuluk/miniconda3/envs/xcodec2-infer-lib-buildtest/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Some weights of XCodec2Model were not initialized from the model checkpoint at /home/uetuluk/hkust-audio-models/models/xcodec2 and are newly initialized: ['generator.backbone.transformers.0.att.rotary_embed.freqs', 'generator.backbone.transformers.1.att.rotary_embed.freqs', 

XCodec2Model(
  (semantic_model): Wav2Vec2BertModel(
    (feature_projection): Wav2Vec2BertFeatureProjection(
      (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=160, out_features=1024, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Wav2Vec2BertEncoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-23): 24 x Wav2Vec2BertEncoderLayer(
          (ffn1_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (ffn1): Wav2Vec2BertFeedForward(
            (intermediate_dropout): Dropout(p=0.0, inplace=False)
            (intermediate_dense): Linear(in_features=1024, out_features=4096, bias=True)
            (intermediate_act_fn): SiLU()
            (output_dense): Linear(in_features=4096, out_features=1024, bias=True)
            (output_dropout): Dropout(p=0.0, inplace=False)
          )
          (self_attn_layer_norm): LayerNorm((1024,),

tts

In [3]:
import torch
from safetensors.torch import load_file

loaded_data = load_file("speech_tokens.safetensors", device="cpu")  # Load onto CPU

speech_tokens = loaded_data["speech_tokens"]

gen_wav = Codec_model.decode_code(speech_tokens)

In [4]:
import numpy as np
import wave

# Assuming gen_wav is a PyTorch tensor
wav_data = gen_wav[0, 0, :].cpu().numpy()

# Normalize the data to int16 range if needed
wav_data = np.int16(wav_data / np.max(np.abs(wav_data)) * 32767)

# Write the WAV file using the wave module
with wave.open("test.wav", "w") as wav_file:
    wav_file.setnchannels(1)        # Mono channel
    wav_file.setsampwidth(2)        # 2 bytes per sample (16-bit)
    wav_file.setframerate(16000)    # Sample rate
    wav_file.writeframes(wav_data.tobytes())

cloning

In [5]:
import wave
import struct
import torch

def read_wav_native(filename):
    # Open the WAV file using the built-in wave module.
    with wave.open(filename, 'rb') as wav_file:
        sr = wav_file.getframerate()
        n_channels = wav_file.getnchannels()
        sample_width = wav_file.getsampwidth()  # in bytes
        n_frames = wav_file.getnframes()
        raw_data = wav_file.readframes(n_frames)
    
    # Unpack the raw bytes depending on sample width.
    if sample_width == 1:
        # 8-bit WAV files are stored as unsigned bytes.
        fmt = f'{n_frames * n_channels}B'
        data = struct.unpack(fmt, raw_data)
        # Convert to float in range [-1, 1]
        data = [(sample - 128) / 128.0 for sample in data]
    elif sample_width == 2:
        # 16-bit WAV files are stored as signed shorts (little-endian).
        fmt = f'<{n_frames * n_channels}h'
        data = struct.unpack(fmt, raw_data)
        # Normalize by the maximum value for 16-bit audio.
        data = [sample / 32768.0 for sample in data]
    else:
        raise ValueError(f"Unsupported sample width: {sample_width}")

    # Reshape data into a list of frames.
    # If the audio is multi-channel, each frame will be a list of channel samples.
    if n_channels > 1:
        frames = [
            [data[i * n_channels + ch] for ch in range(n_channels)]
            for i in range(n_frames)
        ]
    else:
        frames = list(data)

    # Convert the list to a torch tensor and add a batch dimension.
    prompt_wav = torch.tensor(frames, dtype=torch.float32)
    prompt_wav = prompt_wav.unsqueeze(0)  # Now shape is (1, n_frames, channels) for multi-channel or (1, n_frames) for mono.
    
    return prompt_wav, sr

# Example usage:
prompt_wav, sr = read_wav_native("test.wav")
print("Sample rate:", sr)
print("Tensor shape:", prompt_wav.shape)

Sample rate: 16000
Tensor shape: torch.Size([1, 351680])


In [6]:
vq_code_prompt = Codec_model.encode_code(input_waveform=prompt_wav)
print("Prompt Vq Code Shape:", vq_code_prompt.shape )   

vq_code_prompt = vq_code_prompt[0,0,:]

Prompt Vq Code Shape: torch.Size([1, 1, 1098])
